### 책소개 / 핵심문장 을 이용한 '생성문장', '해시태그' 생성 프롬프트 및 랭체인

In [1]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from langchain_upstage import ChatUpstage
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os
import time
load_dotenv()
API_KEY = os.getenv("API_KEY")

# to generate sentence
sentence_system_template = SystemMessagePromptTemplate.from_template("""
당신은 작가의 문체를 학습하고 독자에게 전달하고자 하는 메시지를 생성하는 전문가입니다.
주어진 책의 소개글과 핵심 문장을 기반으로 **작가의 문체를 반영한 짧은 메시지를 작성**하세요.                                                                  

작성 조건 : 
- 30자 이내로 작성하세요.
- 작가의 이름을 포함하지 마세요.
- 난해한 표현을 피하세요.
- **문어체를 사용하세요.**                                                                                                                                                                                                                                                                                                                                                                                                                        
""")
sentence_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다. 문체를 학습하세요.
{text_key_sentences}

위의 내용을 기반으로, 독자에게 작가가 전달하고 싶은 메시지를 작성해주세요.  
단, 이 작가의 문체를 반영해야 합니다.
**책의 핵심 내용을 요약하여 담고 있어야 합니다.**
""")
sentence_prompt_template = ChatPromptTemplate.from_messages([sentence_system_template, sentence_user_template])



# to generate hashtag
hashtag_system_template = SystemMessagePromptTemplate.from_template("""
당신은 책에 대한 정보를 바탕으로 적절한 해시태그를 생성하는 전문가입니다.
책의 소개글과 핵심 문장들을 분석하여 관련된 해시태그만 10개 작성하세요.
해시태그의 설명은 필요없습니다.                                                                   
""")

hashtag_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다.
{text_key_sentences}

위 내용을 바탕으로 관련된 해시태그 딱 10개만 작성하세요.
- 책의 핵심 주제와 감성을 반영해야 합니다.
- 일반적인 태그 (#독서, #책추천)나 작가이름(#한강에세이), 카테고리(#힐링)보다 책의 개성을 보여주는 태그를 포함하세요.   
- 감성적 태그 (#마음에_와닿는_책), 장르 태그 (#심리치유), 실용적 태그 (#자기계발서) 등을 혼합하세요. 
- 해시태그 앞에는 반드시 `#`을 붙이고, 공백 없이 작성하세요.
- **절대 `_`(언더바)를 사용하지 마세요.** (예: `#책_추천` → `#책추천`으로 수정)
- **출력 시 `_`(언더바)가 포함된 해시태그가 나오면 실패한 결과로 간주합니다.**  
                                                                                                                                                                                             
**❌ 피해야 할 예시**                                                                 
- #일의_감각 (잘못된 형식, `_` 포함)
- #브랜드_스토리 (잘못된 형식, `_` 포함)                                                                

**✅ 올바른 예시**
- #일의감각 (공백 없이 작성)
- #브랜드스토리 (공백 없이 작성)                                                                                                                                                                                                                                                 
""")

hashtag_prompt_template = ChatPromptTemplate.from_messages([hashtag_system_template, hashtag_user_template])



# to generate letter
letter_system_template = SystemMessagePromptTemplate.from_template("""
당신은 이 책의 작가입니다. 이 책을 찾은 독자들에게 책을 추천하는 편지를 씁니다.

편지 작성 기준 :
- **반드시 200자이내**로 작성하세요.(300자를 초과하면 실패한 결과로 간주합니다.)
- 마치 독자와 직접 대화하는 듯한 편지 형식으로 작성하세요.
- 독자가 이 책을 읽고 싶어지도록 **기대감을 주는 내용**이어야 합니다.
- **작가의 인사나, 소개는 꼭 작성하지 않습니다.**                                                                                                                                                                                                                                                                                                                                            
""")

letter_user_template = HumanMessagePromptTemplate.from_template("""
아래 [text_description]는 이 책의 소개글 입니다. 제 3자의 시선으로 이 책에 대해 알려주는 글 입니다.
{text_description}

아래 [text_key_sentences]는 이 책의 본문에서 추출한 핵심문장들입니다. 작가가 직접 쓴 문장이므로 문체가 담겨있습니다.
{text_key_sentences}

위 내용과 문체를 학습한 후, 이 책의 작가로서 독자에게 강렬한 편지를 남겨주세요.
- 책을 읽기 전 독자가 책을 기대할 수 있게 감정을 전달하세요.
- 책의 **분위기와 작가 특유의 문체를 자연스럽게 반영**하세요.
- **꼭 독자에게 직접 말하는 듯한 어조를 사용하세요.**
- **반드시 200자이내**로 작성하세요.(300자를 초과하면 실패한 결과로 간주합니다.)
- **작가의 인사나, 소개는 꼭 작성하지 않습니다.** 
                                                                
**❌ 피해야 할 예시** 
- 안녕하세요, 독자님.
- 안녕하세요, 독자에게.
- 안녕하세요                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
""")

letter_prompt_template = ChatPromptTemplate.from_messages([letter_system_template, letter_user_template])




# 2. Initialize the LLM (using OpenAI GPT as an example)
llm = ChatUpstage(api_key=API_KEY, model_name="solar-pro", temperature=0.7)

# 3. Create the LangChain using the template and LLM
sentence_chain = LLMChain(llm=llm, prompt=sentence_prompt_template)
hashtag_chain = LLMChain(llm=llm, prompt=hashtag_prompt_template)
letter_chain = LLMChain(llm=llm, prompt=letter_prompt_template)

/var/folders/pv/stmwcx8d7dsf_j4ytc5gp5780000gn/T/ipykernel_3678/1694209679.py:113: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sentence_chain = LLMChain(llm=llm, prompt=sentence_prompt_template)


In [2]:
# 4. Function to process the text and get the result
def get_author_message(text_description, text_key_sentences):
    time.sleep(1)
    return sentence_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

def get_hashtags(text_description, text_key_sentences):
    time.sleep(1)
    return hashtag_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

def get_letter(text_description, text_key_sentences):
    time.sleep(1)
    return letter_chain.run({
            "text_description": text_description,
            "text_key_sentences": text_key_sentences
        }
    )

# 5. Function to process multiple books
def process_multiple_books(text_description, text_key_sentences, isbn_list):
    results = {}
    for idx, (text_ds, text_ks) in enumerate(zip(text_description, text_key_sentences)):
        print(f"Processing book {idx}...")
        message = get_author_message(text_ds, text_ks)
        hashtags = get_hashtags(text_ds, text_ks)
        letter = get_letter(text_ds, text_ks)
        results[f"Book {idx}"] = {
            "message": message,
            "hashtags": hashtags,
            "letter": letter,
            "isbn": isbn_list[idx]
        }

        # 파일 저장 경로 설정
        file_path = "notebook/data"
        os.makedirs(file_path, exist_ok=True)  # 디렉토리가 없으면 생성

        # JSON 파일로 저장
        output_file = os.path.join(file_path, "llm_output.json")
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)

        print(f"Results have been saved to {output_file}")

    return results

### 책 데이터로 부터 리스트 생성

In [3]:
import json
import re

# JSON 데이터 로드
with open('../data/scraping/filtered_book_unique.json', "r", encoding="utf-8") as file:
    data = json.load(file)

print(len(data))
# description과 key_sentences를 통합한 리스트 생성
description_list = []
key_sentences_list = []
isbn_list = []
isbn_loss_idx = []

for idx, el in enumerate(data[100:105]):
    # description과 key_sentences를 가져옴
    description = el.get('description', '')  # description이 없으면 빈 문자열
    key_sentences = el.get('key_sentences',)  # key_sentences가 없으면 빈 문자열
    isbn_str = str(el.get('isbn', ''))  # isbn13을 문자열로 변환

    # 숫자만 추출하고 빈 문자열인 경우 처리
    isbn_numeric = re.sub(r"\D", "", isbn_str)
    if isbn_numeric:  # 숫자가 있는 경우에만 int 변환
        isbn13 = int(isbn_numeric)
    else:
        isbn13 = None
        isbn_loss_idx.append(idx)


    # 리스트에 추가
    description_list.append(description)
    key_sentences_list.append(key_sentences)
    isbn_list.append(isbn13)

# 결과 확인
print(f"총 {len(key_sentences_list)}개의 항목이 생성되었습니다.")
print(description_list[:1])
print(key_sentences_list[:1])  # 샘플 출력

1781
총 5개의 항목이 생성되었습니다.
['★ 누적 조회수 9억 뷰, 100만 크리에이터 ‘얼미부부’ 첫 에세이 ★★ SNS 웃음 폭탄 얼미부부만의 행복 소환 노하우 대방출 ★“우리 자기들 더 행복해질 준비해요 아주 그냥!”잇몸 마르게 웃다가도 갑자기 마음이 찡해지고느닷없이 행복을 깨닫게 된다헌팅 포차에서 개그 배틀을 하다 서로에게 반하고 11년 사귄 후 결혼한 커플. 유쾌한 티키타카와 쉴 새 없이 서로를 웃게 만드는 일상을 담은 영상들로 단숨에 100만 크리에이터가 되었다. 얼미부부의 틱톡과 유튜브, 릴스에는 “얼미부부 영상을 보면 행복해져요!”, “결혼하면 얼미부부처럼 살고 싶어요”, “일반인 구독해서 보는 건 처음이에요. 너무 웃겨요” 같은 댓글이 폭주한다. 그들만의 행복 에너지는 3년 만에 누적 조회수 9억 뷰를 만들었다.구독자들에게 웃음을 전하는 얼미부부에게도 힘든 날은 있었다. 개그맨 공채 다섯 번 탈락, 대학 입시 포기, 20만 원 반지하 월셋방과 가수 오디션 고배까지……. 우울한 나날이 속절없이 밀려왔지만 인생의 고난이 둘의 행복을 가로막지는 못했다. 실패하고 넘어져도 다시 시도하고, 때로는 깨끗하게 포기하는 용기를 내야 할 때 든든하게 믿고 지켜봐준 서로가 있었기 때문이다. 무엇보다 평범한 일상에서도 서로의 웃음 버튼을 눌러주는 노력을 게을리 한 적이 없다. 얼미부부는 서로 오래 사랑하고 행복을 놓치지 않는 비밀을 “혼자 행복한 사람이 같이 있을 때도 행복하다”라는 마음가짐에서 찾는다.자기들 행복만큼이나 구독자들의 행복에도 진심인 이들은 용기 내어 자신들의 이야기를 전하기로 했다. 얼미부부표 행복 소환 에세이 『우리는 날마다 조금씩 행복해진다』는 행복은 그리 멀지 않다는 것, 행복도 습관처럼 언제든 연습할 수 있다는 메시지를 진심을 담아 전하고 있다. 대한민국 최애 부부로 등극한 얼미부부와 함께 1일 1행복을 소환하는 주문을 외쳐보자. “엉망진창 같아도 결국엔 해피엔딩이야!”']
['그래서 우리는 불행을 건너뛰기로 했다. 우리는 우울한 날에

In [4]:
print(len(data))
print(len(description_list))
print(len(key_sentences_list))
print(len(isbn_list))
print(isbn_list[0])
print(len(isbn_loss_idx))
# print(data[4884])

1781
5
5
5
9788901284521
0


### 예외처리 및 실행

- 1시간당 750개 처리

In [5]:

# Example usage
if __name__ == "__main__":

    if len(description_list) != len(data) or len(key_sentences_list) != len(data):
        print("Warning: The length of text_description and text_key_sentences is not the same.")
        min_length = min(len(description_list), len(key_sentences_list))
        print(f"Processing only the first {min_length} items.")
    else:
        min_length = len(description_list)


    try:
        text_description = description_list[:min_length]
        text_key_sentences = key_sentences_list[:min_length]
    except Exception as e:
        print(f"An error occurred while processing the books: {e}")

    results = process_multiple_books(text_description, text_key_sentences, isbn_list)

    output_filename = "book_information.json"  # 저장할 파일 이름
    with open(output_filename, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print(f"✅ JSON 파일 저장 완료: {output_filename}")

    
    for book, data in results.items():
        print(f"{book} 메시지: {data['message']}")
        print(f"{book} 해시태그: {data['hashtags']}")
        print(f"{book} 작가의편지: {data['letter']}",'\n')

Processing only the first 5 items.
Processing book 0...


/var/folders/pv/stmwcx8d7dsf_j4ytc5gp5780000gn/T/ipykernel_3678/515208469.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return sentence_chain.run({


Results have been saved to notebook/data/llm_output.json
Processing book 1...
Results have been saved to notebook/data/llm_output.json
Processing book 2...
Results have been saved to notebook/data/llm_output.json
Processing book 3...
Results have been saved to notebook/data/llm_output.json
Processing book 4...
Results have been saved to notebook/data/llm_output.json
✅ JSON 파일 저장 완료: book_information.json
Book 0 메시지: 행복은 습관입니다. 연습하세요.
Book 0 해시태그: #행복소환 #일상행복 #행복습관 #행복노하우 #연애꿀팁 #소통법 #실패극복 #도전정신 #자기이해 #에세이추천
Book 0 작가의편지: 책장을 넘기는 순간, 당신의 인생이 해피엔딩임을 깨닫게 될 거예요. 우리도 해냈거든요! 함께 웃고, 울며 행복을 찾아나서요. 이 책을 통해 당신의 인생도 조금씩 행복해지길 바랍니다. 

Book 1 메시지: 좋아지는 일들이 더 많았으면 합니다.
Book 1 해시태그: #계절산문 #박준시인 #서정적산문 #사계절의기억 #좋은것들과함께 #좋아지는일들 #은근슬쩍스스로를좋아할수도있을테니까요 #한철동안의아름다운모습 #아름다운우리의가을날 #서로에게번화했으므로시간은우리를웃자라게했습니다
Book 1 작가의편지: 살아가면서 좋아지는 일들이 더 많았으면 합니다. 좋은 것들과 함께라면, 은근슬쩍 스스로를 좋아할 수 있을 테니까요. 이 계절, 함께 산책하지 않으시겠습니까? 

Book 2 메시지: 좋은 대화는 숙성된 진심에서 나옵니다.
Book 2 해시태그: #서정적인대화
#풍부한표현력
#진심을나눈인생
#상어식대화
#고래식대화
#이불킥의기억
#입장
#숙성의시간